# 사전에 없는 단어의 문제

```
코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤
전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.

→

<unk>는 2019년 12월 중국 <unk>에서 처음 발생한 뒤
전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.
```

 만약 위 문장을 영문으로 번역해야 한다면 어떨까요? 핵심인 단어 `코로나바이러스`와 `우한`을 모른다면 제대로 해낼 수 있을 리가 없습니다. 이를 **OOV(Out-Of-Vocabulary)** 문제라고 합니다. 이처럼 **새로 등장한(본 적 없는) 단어에 대해 약한 모습**을 보일 수밖에 없는 기법들이기에, 이를 해결하고자 하는 시도들이 있었습니다. 그리고 그것이 우리가 다음 스텝에서 배울, ***Wordpiece Model***이죠!

*Wordpiece Model(WPM)* 은 우리가 접한 적이 있는 아이디어를 기반으로 만들어졌습니다. 두 단어 `preview`와 `predict`를 보면 접두어인 `pre`가 공통되고 있죠? `pre`가 들어간 단어는 주로 **"미리", "이전의"** 와 연계되는 의미를 가지고 있습니다. 컴퓨터도 두 단어를 따로 볼 게 아니라 `pre+view`와 `pre+dict`로 본다면 학습을 더 잘 할 수 있지 않을까요?

이처럼 한 단어를 여러 개의 Subword의 집합으로 보는 방법이 WPM입니다. WPM의 원리를 알기 전, 먼저 알아야 할 것이 바로 **Byte Pair Encoding(BPE)** 입니다.

# Byte Pair Encoding(BPE)

---

BPE 알고리즘이 고안된 것은 1994년입니다. 그때는 자연어 처리에 적용하기 위해서가 아니라 데이터 압축을 위해서 생겨났었죠. 데이터에서 **가장 많이 등장하는 바이트 쌍(Byte Pair)** 을 새로운 단어로 치환하여 압축하는 작업을 반복하는 방식으로 동작합니다. 예시는 아래와 같습니다.

```
aaabdaaabac # 가장 많이 등장한 바이트 쌍 "aa"를 "Z"로 치환합니다.
→ 
ZabdZabac   # "aa" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.
Z=aa        # 그다음 많이 등장한 바이트 쌍 "ab"를 "Y"로 치환합니다.
→ 
ZYdZYac     # "ab" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.
Z=aa        # 여기서 작업을 멈추어도 되지만, 치환된 바이트에 대해서도 진행한다면
Y=ab        # 가장 많이 등장한 바이트 쌍 "ZY"를 "X"로 치환합니다.
→ 
XdXac
Z=aa
Y=ab
X=ZY       # 압축이 완료되었습니다!
```

아주 직관적인 알고리즘이죠? 이를 토큰화에 적용하자고 제안한 것은 2015년이었습니다. 모든 단어를 문자(바이트)들의 집합으로 취급하여 자주 등장하는 문자 쌍을 합치면, 접두어나 접미어의  의미를 캐치할 수 있고, 처음 등장하는 단어는 문자(알파벳)들의 조합으로 나타내어 **OOV 문제를 완전히 해결**할 수 있다는 것이죠!

비교적 최근의 기술을 소개해드리는 만큼 논문을 함께 첨부합니다.

- [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/pdf/1508.07909.pdf)

위 논문은 Python 소스 코드를 함께 제공해 주어 간편하게 실습을 해 볼 수 있습니다. 논문에서 제공해 주는 예제로 동작 방식을 자세히 들여다보죠!

In [1]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 3.9 MB/s 


In [2]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

In [7]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7fe9fd49ea10>)

In [8]:
train_df = pd.read_csv('IMDb_Reviews.csv')

In [9]:
train_df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [10]:
print('리뷰 갯수 : ', len(train_df))

리뷰 갯수 :  50000


In [11]:
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

In [12]:
corpus = 'imdb_review.txt' # 입력 corpus
prefix = 'imdb' # 저장할 단어장 이름
vocab_size = 5000
spm.SentencePieceTrainer.Train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size}" +
    "--model_type=bpe" +
    "--max_sentence_length=999999" #문장 최대 길이
)

In [13]:
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting = csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
738,▁instead,-8.60773
2457,▁suddenly,-9.93209
411,▁They,-8.01949
3376,▁contrast,-10.39330
1211,▁huge,-9.10864
3465,▁Max,-10.43480
2332,af,-9.87183
2907,▁Terr,-10.16210
3991,▁alcohol,-10.72860
2784,▁Life,-10.10150


In [14]:
len(vocab_list)

5000

In [15]:
sp = spm.SentencePieceProcessor()
vocab_file = 'imdb.model'
sp.load(vocab_file)

True

In [16]:
lines = [
         "I didn't at all think of it this way.",
         "I have waited a long time for someone to film"
]

for line in lines:
    print(line)
    print(sp.encode_as_pieces(line)) # 문장을 입력하면 서브워드 시퀀스로 변환
    print(sp.encode_as_ids(line)) #문장을 입력하면 정수 시퀀스로 변환
    print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[16, 250, 11, 15, 56, 52, 146, 9, 17, 20, 139, 6]

I have waited a long time for someone to film
['▁I', '▁have', '▁wait', 'ed', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[16, 45, 1401, 27, 7, 338, 82, 28, 543, 10, 30]



In [17]:
sp.GetPieceSize() # 단어집합의 크기

5000

In [18]:
sp.IdToPiece(120) # 정수로부터 매핑되는 서브워드 변환

'▁will'

In [20]:
sp.PieceToId('▁will')

120

In [21]:
sp.DecodeIds([16, 250, 11, 15, 56, 52, 146, 9, 17, 20, 139, 6])
# 정수 시퀀스를 sp.DecodeIds의 정수 시퀀스에 입력

"I didn't at all think of it this way."

In [22]:
sp.DecodePieces(['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.'])

"I didn't at all think of it this way."

In [23]:
import pandas as pd
import sentencepiece as spm
import urllib.request
import csv

In [24]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7fe9fd23c210>)

In [25]:
naver_df = pd.read_table('ratings.txt')
naver_df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [26]:
print('리뷰 갯수 :', len(naver_df))

리뷰 갯수 : 200000


In [27]:
print(naver_df.isnull().values.any()) # null값 존재 확인

True


In [28]:
# Null값이 존재하는 행 제거
naver_df = naver_df.dropna(how='any')
# Null값이 존재하는지 확인
print(naver_df.isnull().values.any())

False


In [29]:
print('리뷰 갯수 :', len(naver_df))

리뷰 갯수 : 199992


In [36]:
# naver data 'document'컬럼을 txt에 새로 만드세요.
with open('naver_document.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))



In [37]:
# sentencePieceTrainer로 학습
corpus = 'naver_document.txt' # 입력 corpus
prefix = 'document' # 저장할 단어장 이름
vocab_size = 5000
spm.SentencePieceTrainer.Train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size}" +
    "--model_type=bpe" +
    "--max_sentence_length=999999" #문장 최대 길이
)

In [38]:
# vocab을 불러오기
vocab_list = pd.read_csv('document.vocab', sep='\t', header=None, quoting = csv.QUOTE_NONE)


In [39]:
# vocab모델이 있는지 확인
vocab_list.sample(10)

,0,1
3673,▁억지스럽,-10.05020
3447,▁낮아서,-9.94835
2000,▁나온다,-9.30677
2155,▁폭,-9.38412
2815,▁어이없는,-9.66973
4468,벙,-11.22410
3026,▁보는게,-9.76046
3522,▁밋밋,-9.97568
2318,지마,-9.45589
4445,툰,-11.16520


In [40]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 55.7 MB/s 


In [41]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt